In [1]:
from python_speech_features import mfcc
from python_speech_features import delta
from python_speech_features import logfbank
import pandas as pd
import numpy as np
from scipy.signal import get_window
import scipy.fftpack as fft

In [2]:

def emphasized_audio(audio, alpha=0.97):
    emphasized_audio = np.append(audio[0], audio[1:] - alpha * audio[:-1])
    return emphasized_audio


def normalize_audio(audio):
    audio = audio / np.max(np.abs(audio))
    return audio


def frame_audio(audio, FFT_size=2048, hop_size=10, sample_rate=44100):
    # hop_size in ms
    audio = np.pad(audio, int(FFT_size / 2), mode='reflect')
    frame_len = np.round(sample_rate * hop_size / 1000).astype(int)
    frame_num = int((len(audio) - FFT_size) / frame_len) + 1
    frames = np.zeros((frame_num, FFT_size))

    for n in range(frame_num):
        frames[n] = audio[n*frame_len:n*frame_len+FFT_size]

    return frames


def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)


def met_to_freq(mels):
    return 700.0 * (10.0**(mels / 2595.0) - 1.0)


def get_filter_points(fmin, fmax, mel_filter_num, FFT_size, sample_rate=44100):
    fmin_mel = freq_to_mel(fmin)
    fmax_mel = freq_to_mel(fmax)

    # print("MEL min: {0}".format(fmin_mel))
    # print("MEL max: {0}".format(fmax_mel))

    mels = np.linspace(fmin_mel, fmax_mel, num=mel_filter_num+2)
    freqs = met_to_freq(mels)
    # f(m-1)和f(m)、f(m+1)分别对应第m个滤波器的起始点、中间点和结束点。大家一定要注意的一点是，这里的f(m)对应的值不是频率值，而是对应的sample的索引！比如，我们这里最大频率是22050 Hz, 所以22050Hz对应的是第513个sample，即频率f所对应的值是f/fs*NFFT
    return np.floor((FFT_size + 0.5) / sample_rate * freqs).astype(int), freqs


def get_filters(filter_points, FFT_size):
    filters = np.zeros((len(filter_points)-2, int(FFT_size/2+1)))

    for n in range(len(filter_points)-2):
        # 相比于原kaggle代码，增加了`endpoint=False`参数
        filters[n, filter_points[n]: filter_points[n + 1]] = np.linspace(
            0, 1, filter_points[n + 1] - filter_points[n], endpoint=False)
        filters[n, filter_points[n + 1]: filter_points[n + 2]] = np.linspace(
            1, 0, filter_points[n + 2] - filter_points[n + 1], endpoint=False)

    return filters


def dct(dct_filter_num, filter_len):
    basis = np.empty((dct_filter_num, filter_len))
    basis[0, :] = 1.0 / np.sqrt(filter_len)

    samples = np.arange(1, 2 * filter_len, 2) * np.pi / (2.0 * filter_len)

    for i in range(1, dct_filter_num):
        basis[i, :] = np.cos(i * samples) * np.sqrt(2.0 / filter_len)

    return basis



In [24]:

def get_cepstral_coefficents(filePath, mel_filter_num=10):
    df = pd.read_csv(filePath)
    y = df['ACC_X'].to_numpy()
    t = df['Time'].to_numpy()
    sample_rate = int(1/np.mean(np.diff(t)))

    # nomalize和emphazie 区别？
    # audio = normalize_audio(y)
    audio = emphasized_audio(y, alpha=0.97)

    hop_size = 15  # ms 以这个作为帧的间隔，相当于stride
    FFT_size = 2048

    audio_framed = frame_audio(
        audio, FFT_size=FFT_size, hop_size=hop_size, sample_rate=sample_rate)

    # 加窗 跟hamming？
    window = get_window("hann", FFT_size, fftbins=True)
    audio_win = audio_framed * window

    # 进行STFT
    # 这种转置再转置的原因：在音频信号处理中，更习惯将时间窗口放在第一维
    audio_winT = np.transpose(audio_win)  # audio_winT.shape:(2048,133)

    audio_fft = np.empty(
        (int(1 + FFT_size // 2), audio_winT.shape[1]), dtype=np.complex64, order='F')

    # 对每一帧进行fft
    for n in range(audio_fft.shape[1]):
        audio_fft[:, n] = fft.fft(audio_winT[:, n], axis=0)[
            :audio_fft.shape[0]]

    audio_fft = np.transpose(audio_fft)  # audio_fft.shape:(133, 1025)

    audio_power = np.square(np.abs(audio_fft))

    freq_min = 0
    freq_high = sample_rate / 2  # 奈奎斯特频率
    # mel_filter_num = 18  # 参数可调

    filter_points, mel_freqs = get_filter_points(
        freq_min, freq_high, mel_filter_num, FFT_size, sample_rate=44100)

    filters = get_filters(filter_points, FFT_size)

    enorm = 2.0 / (mel_freqs[2:mel_filter_num+2] - mel_freqs[:mel_filter_num])
    filters *= enorm[:, np.newaxis]  # np.newaxis用于增加一个维度，可以与filter的向量元素做内积

    audio_filtered = np.dot(filters, np.transpose(audio_power))
    # audio_log = 10.0 * np.log10(audio_filtered)  
    audio_log = np.log10(audio_filtered) 

    dct_filter_num = 13
    dct_filters = dct(dct_filter_num, mel_filter_num)
    cepstral_coefficents = np.dot(dct_filters, audio_log)
    # print('logfbank:',cepstral_coefficents)
    return cepstral_coefficents

In [26]:
filePath = './output_cropped/20240112_142401.csv'
df = pd.read_csv(filePath)
sig = df['ACC_X'].to_numpy()
t = df['Time'].to_numpy()
rate = int(1/np.mean(np.diff(t)))
# (rate,sig) = wav.read("english.wav")
mfcc_feat = mfcc(sig,rate,winstep=0.015,numcep=13,nfilt=26,nfft=2048,ceplifter=0,winfunc=np.hanning)
mfcc_feat_T = np.transpose(mfcc_feat)
print(mfcc_feat_T)
# d_mfcc_feat = delta(mfcc_feat, 2)
fbank_feat = logfbank(sig,rate)

# print('library:',fbank_feat[1:3,:])


[[ 5.58032071e+00  6.66232243e+00  6.31704703e+00 ...  5.22991284e+00
   6.93591647e+00  4.53956245e+00]
 [-5.82656650e+00 -7.44399746e+00 -4.07573693e+00 ...  3.30332026e-01
  -4.31003598e+00  5.42789232e-01]
 [ 1.54607152e+00 -9.33230512e-01  2.84592955e+00 ... -1.57403305e+00
   1.53067890e+00  5.29194560e+00]
 ...
 [-6.76872946e-01  9.91061584e-01 -5.95738854e-01 ... -2.53866724e-01
   5.74545388e-01  8.51127529e-01]
 [ 9.21291867e-01  1.55775906e-01  1.39793521e-03 ... -4.94758185e-01
  -1.37871457e-02 -1.64686787e-01]
 [-1.36767729e-01  3.87471649e-01  5.18921891e-01 ... -2.67114889e-01
  -3.89496869e-01  3.56080496e-02]]


In [27]:
a = get_cepstral_coefficents(filePath,mel_filter_num=26)
a

array([[ 1.85467115e+01,  1.87459661e+01,  1.88940370e+01, ...,
         9.82854643e+00,  9.76027664e+00,  9.63635349e+00],
       [-1.25994003e+00, -1.32719134e+00, -1.34608307e+00, ...,
         3.28889551e+00,  3.32316200e+00,  3.37853059e+00],
       [ 5.17202689e-01,  3.13400796e-01,  2.17153549e-01, ...,
         1.19968036e+00,  1.27433625e+00,  1.41530998e+00],
       ...,
       [-1.03467313e+00, -7.77294359e-01, -6.28959963e-01, ...,
        -5.53570309e-01, -6.66319563e-01, -8.50148101e-01],
       [-5.69074181e-01, -6.17414943e-01, -6.48430215e-01, ...,
        -4.29274830e-02, -1.64234390e-02,  2.06089533e-02],
       [ 3.67318523e-01,  1.40230935e-01,  1.52986089e-02, ...,
         9.37923350e-01,  1.02157892e+00,  1.17514513e+00]])